In [ ]:
import sys
!{sys.executable} -m pip install sympy
!{sys.executable} -m pip install plotly

# Task 1 (40 points):
Based on Planck’s law for the radiance of the Sun, assuming a temperature $T$ of $5775\text{K}$, create four plots, which show
A. The spectral irradiance versus photon wavelength
B. The spectral irradiance versus photon energy
C. The photon flux versus photon wavelength
D. The photon flux versus energy.

You can convert the radiance of the object to the irradiance by multiplying the radiance by the steradian angle of the Sun’s disc as it is observed on Earth. That value is $6.794\cdot10^{-5}\text{sr}$. At the end of Task 1, you should have four plots that look similar to the ones on the PV Lighthouse web site:
https://www2.pvlighthouse.com.au/resources/courses/altermatt/The%20Solar%20Spectrum/Intensity%20or%20photon%20flux.aspx

The axis labels should match the ones on PV Lighthouse, except that you should not convert the photon flux to
$\frac{\text{mA}}{\text{cm}^2\cdot\text{eV}}$, but leave it as $\frac{\text{Number of photons}}{\text{s}\cdot\text{m}^2\cdot\text{eV}}$.

In [19]:
from numpy import exp, linspace
import QMFunctions as qm
from QMFunctions import c, q

from plotly.express import scatter, line
from pandas import DataFrame #easiest way to get plotly to display axis labels is if the data is put in a dataframe with labeled columns

sun_steradian_angle = 6.794*10**(-5) # in sr

A. The spectral irradiance versus photon wavelength

In [20]:
def irradiance_wavelength(T, Lambda):
  q = qm.q
  h = q*qm.h
  k_b = q*qm.k_b
  #h and k_b multiplied by q = 1.6022*10**(-19) to convert the eV's in their units to Joules, so the irradiance has units of Watts per m^3
  irradiance = (2 * h * (c**2)/((exp(h * c/(Lambda * k_b * T)) - 1) * Lambda**5)) * sun_steradian_angle
  return irradiance*10**(-9) # convert units from W/m^3 to W/(m^2*nm) to match units used on pvlighthouse website

In [27]:
wavelength = linspace(100, 3000, num = 2900) # in nanometers
irradiance = []

for i in wavelength:
  irradiance.append(irradiance_wavelength(T = 5775, Lambda = i * 10 ** (-9)))

irradiance_wavelength_df = DataFrame({'Wavelength(nm)': wavelength, 'Irradiance(W/(m^2 nm))': irradiance})

scatter(irradiance_wavelength_df, x = "Wavelength(nm)", y = 'Irradiance(W/(m^2 nm))')

B. The spectral irradiance versus photon energy

Frequency form of Planck's radiation law is:
$B_f(f, T) = \frac{2hf^3}{c^2(\exp(\frac{hf}{k_b T}) - 1)}$
To rewrite in terms of energy, we can use $E=hf$ to replace the $f$'s with $\frac{E}{h}$, and then multiply the whole thing by $\frac{\partial f}{\partial E} = \frac{1}{h}$
 That gives $B_E(E, T) = \frac{2\frac{E^3}{h^3}}{c^2(\exp(\frac{E}{k_b T}) - 1)} = \frac{2E^3}{h^3c^2(\exp(\frac{E}{k_b T}) - 1)}$


In [25]:
def irradiance_energy(T, E):
  h = qm.h
  k_b = qm.k_b
  return (2 * E**3) / ((exp(E / (k_b * T)) - 1) * c**2 * h**3) * sun_steradian_angle * q
  # times q to convert units from (eV/s)/(m^2 eV) to (J/s)/(m^2 eV) = W/(m^2 eV)

In [28]:
energy = linspace(0.01, 5, num = 2900) # in ev
irradiance = []

for i in energy:
  irradiance.append(irradiance_energy(T = 5775, E = i))

irradiance_energy_df = DataFrame({'Energy(eV)': energy, 'Irradiance(W/(eV m^2))': irradiance})

scatter(irradiance_energy_df, x = "Energy(eV)", y = 'Irradiance(W/(eV m^2))')

C. and D. The photon flux versus photon wavelength and versus energy

As worked out in HW 1 problem 2, to convert from power to $\frac{\text{# of photons}}{\text{s}}$ we multiple power by $\frac{\lambda}{hc}$.  And since the units of photon flux are $\frac{\text{# of photons}}{\text{s}\cdot\text{eV}\cdot \text{m}^2}$
 to convert from irradiance in terms to wavelength to flux, we multiply by $\frac{\lambda}{hc}$ and then convert from $\text{nm}$ to $\text{eV}$ by multiplying by $\frac{\lambda}{E}$.

By using $E=hf=\frac{hc}{\lambda} \implies \lambda=\frac{E}{hc} \implies \frac{\lambda}{E} = \frac{1}{hc}$
we get that photon flux $=B_\lambda \cdot \frac{\lambda}{hc}\cdot\frac{1}{hc} = B_\lambda\frac{\lambda}{(hc)^2}$
Also, photon flux $ = B_E \frac{\lambda}{hc} = B_E \frac{\frac{E}{hc}}{hc} = B_E \frac{E}{(hc)^2}$

Photon flux versus photon wavelength:

In [52]:
flux_wavelength_df = irradiance_wavelength_df.\
  rename(columns = {'Irradiance(W/(m^2 nm))': 'Photon Flux(# of photons/(s m^2 eV))'})

flux_wavelength_df['Photon Flux(# of photons/(s m^2 eV))'] = \
  flux_wavelength_df['Photon Flux(# of photons/(s m^2 eV))']*flux_wavelength_df['Wavelength(nm)']*1/(qm.h*q*c)

scatter(flux_wavelength_df, x = "Wavelength(nm)", y = 'Photon Flux(# of photons/(s m^2 eV))')


Photon flux versus photon wavelength:


In [46]:
flux_energy_df = irradiance_energy_df.\
  rename(columns = {'Irradiance(W/(m^2 nm))': 'Photon Flux(# of photons/(s m^2 eV))'})

flux_energy_df['Photon Flux(# of photons/(s m^2 eV))'] = \
  flux_energy_df['Photon Flux(# of photons/(s m^2 eV))']*flux_energy_df['Energy(eV)']*1/(qm.h*q*c)

scatter(flux_energy_df, x = "Energy(eV)", y = 'Photon Flux(# of photons/(s m^2 eV))')
